This file trains and tests the LSTM classifier for the eeg data augmented with the mahalanobis distances of the samples from the event cluster centers.  The modes are sub_event and event.  Sub_event looks at the centers for each hand motion as being zero or one, while the event mode looks at each combination of hand motions

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import Input, Flatten
from keras.layers.recurrent import LSTM
from keras.models import load_model, Model
import keras
from eeg_generator_augmented import DataGenerator

/Users/joenunez/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
batch_size = 50
sample_dim = 32
steps_back = 50
target_dim = (6)
#n_evals cannot exceed sample_dim
n_evals=16
mode='event'
dist='gmm'

generator = DataGenerator(batch_size=batch_size, 
    dim1=sample_dim, target_dim=target_dim, 
    steps_back=steps_back, mode=mode, n_evals=n_evals, dist=dist)

input_shape = generator.get_input_shape()


list_IDs and test_IDs filled


In [10]:
num_epochs = 3

In [11]:
model = Sequential()
model.add(LSTM(1000, input_shape=input_shape))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(target_dim, activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='nadam')
print('model compiled')

model compiled


In [12]:
model.fit_generator(generator=train_generator, epochs=num_epochs, 
                    steps_per_epoch=150, workers=4, use_multiprocessing=True)
model.save("eeg_lstm_sub_gmm16_aug_50_steps.h5")
print('model saved')

Epoch 1/3
150/150 [==============================] - 304s 2s/step - loss: 0.1538
Epoch 2/3
150/150 [==============================] - 317s 2s/step - loss: 0.1248
Epoch 3/3
150/150 [==============================] - 290s 2s/step - loss: 0.1196


In [13]:
X_test, y_test = generator.test_generation(1000)
print('test generated')

test generated


In [14]:
#model = load_model("eeg_lstm_sub_aug_50_steps.h5")
model.summary()
preds = model.predict(X_test)
print('preds made')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 1000)              4388000   
_________________________________________________________________
dense_8 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_9 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 1000)              1001000   
_________________________________________________________________
dense_11 (Dense)             (None, 1000)              1001000   
_________________________________________________________________
dropout_4 (Dropout)          (None, 1000)              0         
__________

Computes the accuracy.  Also possible to compile the model with accuracy as a metric and use evaluate function, but this lets us leverage the round function more easily than putting a specialized layer at the end of the network 

In [15]:
corr=float(0)
for i,p in enumerate(preds):
    if np.array_equal(np.around(p), y_test[i]):
        corr+=1
print(corr/len(preds))

0.87224
